<a href="https://colab.research.google.com/github/ajinkya8484/my_work/blob/master/change_detection_indonesia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from collections import Counter
from scipy.misc import imread, imresize, imsave


def find_vector_set(diff_image, new_size):
   
    i = 0
    j = 0
    num = (((new_size[0] * new_size[1]) / 25).astype(int), 25)
    vector_set = np.zeros(num)
    while i < vector_set.shape[0]:
        while j < new_size[0]:
            k = 0
            while k < new_size[1]:
                block = diff_image[j:j+5, k:k+5]
                feature = block.ravel()
                #print(vector_set.shape)
                #print(block)
                #print(feature)
                vector_set[i, :] = feature
                k = k + 5
            j = j + 5
        i = i + 1
        
    print('\nvector_set shape', vector_set.shape)
            
    mean_vec = np.mean(vector_set, axis=0)
    vector_set = vector_set - mean_vec
    
    return vector_set, mean_vec
    
  
def find_FVS(EVS, diff_image, mean_vec, new):
    
    i = 2 
    feature_vector_set = []
    
    while i < new[0] - 2:
        j = 2
        while j < new[1] - 2:
            block = diff_image[i-2:i+3, j-2:j+3]
            feature = block.flatten()
            feature_vector_set.append(feature)
            j = j+1
        i = i+1
        
    FVS = np.dot(feature_vector_set, EVS)
    FVS = FVS - mean_vec
    print("\nfeature vector space size", FVS.shape)
    return FVS

def clustering(FVS, components, new):
    
    kmeans = KMeans(components, verbose=0)
    kmeans.fit(FVS)
    output = kmeans.predict(FVS)
    count = Counter(output)

    least_index = min(count, key=count.get)
    change_map = np.reshape(output, (new[0].astype(int) - 4, new[1].astype(int) - 4))
    
    return least_index, change_map

   
def find_PCAKmeans(imagepath1, imagepath2):
    
    print('Operating')
    
    image1 = imread(imagepath1)
    image2 = imread(imagepath2)
    print(image1.shape)
    print(image2.shape)

    new_size = np.asarray(image1.shape) / 5 * 5
    image1 = imresize(image1, new_size).astype(np.int16)
    image2 = imresize(image2, new_size).astype(np.int16)

    print(image1.shape)
    print(image2.shape)
    
    diff_image = abs(image1 - image2)   
    imsave('diff.jpg', diff_image)
    print('\nBoth images resized to ', new_size)
        
    vector_set, mean_vec = find_vector_set(diff_image, new_size)
    
    pca = PCA()
    pca.fit(vector_set)
    EVS = pca.components_
        
    FVS = find_FVS(EVS, diff_image, mean_vec, new_size)
    
    print('\ncomputing k means')
    
    components = 3
    least_index, change_map = clustering(FVS, components, new_size)
    
    change_map[change_map == least_index] = 255
    change_map[change_map != 255] = 0
    
    change_map = change_map.astype(np.uint8)
    kernel = np.asarray(((0, 0, 1, 0, 0),
                        (0, 1, 1, 1, 0),
                        (1, 1, 1, 1, 1),
                        (0, 1, 1, 1, 0),
                        (0, 0, 1, 0, 0)), dtype=np.uint8)
    cleanChangeMap = cv2.erode(change_map, kernel)
    imsave("indonesia_changemap.jpg", change_map)
    imsave("indonesia_cleanchangemap.jpg", cleanChangeMap)

    
if __name__ == "__main__":
    a = 'pre event 5000 BW.jpg'
    b = 'post event 5000 BW.jpg'
    #a = 'Andasol_09051987.jpg'
    #b = 'Andasol_09122013.jpg'
    find_PCAKmeans(a, b)

Operating


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


(3500, 3500)
(3500, 3500)
(3500, 3500)
(3500, 3500)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.



Both images resized to  [3500. 3500.]

vector_set shape (490000, 25)

feature vector space size (12222016, 25)

computing k means


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:113: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [2]:
ls


'post event 5000 BW.jpg'  'pre event 5000 BW.jpg'   sample_data/
